In [1]:
!pip install -U langchain langchain-community
!pip install -U langchain-huggingface
!pip install pypdf
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.4 MB/s eta 0:00:00


In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [3]:
vector_db_path = "vectorstores/db_faiss"
pdf_data_path = "/content/data"


In [4]:
def load_and_split_documents(pdf_data_path, chunk_size=512, chunk_overlap=100):
    loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

In [5]:
chunks=load_and_split_documents(pdf_data_path)
print("Văn bản gốc:")
print(pdf_data_path[:])

print("\nSố chunks:", len(chunks))
print("\nChunk đầu tiên:")
print(chunks[0].page_content)
print("\nChunk 2:")
print(chunks[1].page_content)


Văn bản gốc:
/content/data

Số chunks: 19

Chunk đầu tiên:
Tuyển sinh Đại học Xây dựng Hà Nội năm 2025 
Tổng quan về trường 
Đại học Xây dựng Hà Nội (National University of Civil Engineering - NUCE) là một trong 
những cơ sở giáo dục đại học kỹ thuật hàng đầu tại Việt Nam, nổi tiếng với hơn 60 năm hình 
thành và phát triển trong lĩnh vực đào tạo nhân lực chất lượng cao cho ngành xây dựng, 
kiến trúc, kỹ thuật hạ tầng và quản lý đô thị. Thành lập năm 1966, trường đã khẳng định vị

Chunk 2:
kiến trúc, kỹ thuật hạ tầng và quản lý đô thị. Thành lập năm 1966, trường đã khẳng định vị 
thế là trung tâm đào tạo, nghiên cứu khoa học và chuyển giao công nghệ uy tín, đóng góp 
lớn vào sự phát triển cơ sở hạ tầng và kiến trúc đô thị của đất nước. Với đội ngũ giảng viên 
giàu kinh nghiệm, cơ sở vật chất hiện đại và các chương trình hợp tác quốc tế rộng rãi, 
NUCE thu hút hàng nghìn thí sinh đăng ký xét tuyển mỗi năm. 
Chỉ tiêu và phương thức tuyển sinh


In [6]:
def embed_and_save(chunks, vector_db_path, embedding_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    db = FAISS.from_documents(chunks, embedding_model)
    db.save_local(vector_db_path)
    return db

In [7]:
db = embed_and_save(chunks, vector_db_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

from huggingface_hub import snapshot_download
import torch


model_repo = "phamhai/Llama-3.2-3B-Instruct-Frog"
local_model_path = "./Llama-3.2-3B-Instruct-Frog"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
vector_db_path = "vectorstores/db_faiss"


def load_llm(model_repo, local_path):
    model_path = snapshot_download(
        repo_id=model_repo,
        local_dir=local_path,
        local_dir_use_symlinks=False
    )

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16)
    model.to("cuda")

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=1024,
        temperature=0.01,
        do_sample=False,
        return_full_text=False
    )

    llm = HuggingFacePipeline(pipeline=pipe)
    return llm

def create_prompt(template):
    return PromptTemplate(template=template, input_variables=["context", "question"])


def create_qa_chain(prompt, llm, db):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=False,
        chain_type_kwargs={'prompt': prompt}
    )
    return qa_chain


def read_vectors_db():
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    db = FAISS.load_local(
        vector_db_path,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
    return db



db = read_vectors_db()
llm = load_llm(model_repo, local_model_path)

template = """<|im_start|>system
Sử dụng thông tin sau đây để trả lời câu hỏi. Nếu bạn không biết câu trả lời, hãy nói không biết, đừng cố tạo ra câu trả lời.
{context}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

prompt = create_prompt(template)
llm_chain = create_qa_chain(prompt, llm, db)


question = "tuyển sinh đại học xây dựng 2025 yêu cầu gì"
response = llm_chain.invoke({"query": question})
print(response["result"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

pytorch_model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-8-3085675771.py:39: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Để tăng cơ hội trúng tuyển vào Đại học Xây dựng Hà Nội năm 2025, thí sinh cần chuẩn bị kỹ lưỡng cho kỳ thi tốt nghiệp THPT, đặc biệt là ở các môn Toán, Vật lý và Hóa học (tổ hợp A00, A01) hoặc Văn, Toán, Vẽ Mỹ thuật (tổ hợp V00 cho ngành Kiến trúc). Ngoài ra, thí sinh cũng nên tìm hiểu kỹ về các chương trình đào tạo và yêu cầu tuyển sinh của trường để có thể đăng ký phù hợp.


In [9]:
question = input("Nhập câu hỏi của bạn: ")
response = llm_chain.invoke({"query": question})
print("\nTrợ lý AI trả lời:\n", response["result"])


Nhập câu hỏi của bạn: trường có nghành khoa học máy tính không


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Trợ lý AI trả lời:
 Có, trường có ngành Khoa học máy tính. Điểm chuẩn đầu vào của trường là từ 24-27 điểm, tùy thuộc vào ngành học. Trường cũng yêu cầu thí sinh phải thi thêm môn năng khiếu Vẽ Mỹ thuật nếu muốn đăng ký ngành Kiến trúc. Ngoài ra, trường còn có chính sách học bổng và hỗ trợ sinh viên để giúp đỡ những sinh viên có hoàn cảnh khó khăn. Trường cũng có môi trường học tập chuyên nghiệp và cơ hội nghề nghiệp rộng mở cho các sinh viên.


In [10]:
import torch
torch.cuda.is_available()


True